In [1]:
ploting_dir = "/home/uli/Insync/uligenske@gmail.com/Google Drive/Projekte_Uli/Deep_Learning/segmentation/projekte/scripts/plots/2) phantom similarity"

In [2]:
dict_filename_imagetype = {'512x512x078_scale078_version3 (input scaled)': 'gan',
 'GAN589_Aly055_512x512x079_scale079_version3 (finale)': 'gan',
 'GAN_589_ALy055-1mm_LungTable_rem_5357_reg_scale079_512x512x079': 'gan',
 'old_gan_synth_Phm_origSize_20HU+_Intestin_Lung_Table_soft_211003_skaliert078_512x512x078': 'gan',
 'raw_512x512x078_scale078_version3 (input scaled)': 'raw',
 'raw_589_ALy055-1mm_LungTable_rem_5357_reg_scale079_512x512x079': 'raw',
 'raw_Aly055_512x512x079_scale079_version3 (finale)': 'raw',
 'raw_Phm_origSize_20HU+_Intestin_Lung_Table_soft_211003_skaliert078_512x512x078': 'raw'}

In [3]:
import os
import pandas as pd

results_path = "sim_analyis_unzipped_archive"
df = pd.DataFrame()
for root, dirs, files in os.walk(results_path):
    for f in files:
        if f.endswith("sim_metrics.csv"):
            meta_infos_path = root.split("/")[-1].split("_")
            project = root.split("/")[-2]   
            df_temp = pd.read_csv(os.path.join(root, f))
            df_temp["project"] = project
            df_temp["reco"] = f"{meta_infos_path[-5]}_{meta_infos_path[-6]}"
            df_temp["dose"] = meta_infos_path[-7]
            if "version3" in root.split("/")[-1].lower():
                df_temp["lesion_type"] = "replace"
            else:
                df_temp["lesion_type"] = "add"

            for key in dict_filename_imagetype.keys():
                if key in root:
                    df_temp["image_type"] = dict_filename_imagetype[key]

                
            df = df.append(df_temp, ignore_index=True)

In [5]:
import plotly.express as px

import plotly.express as px
import sys
sys.path.append("/home/uli/Insync/uligenske@gmail.com/Google Drive/arbeit/validation_tool/code/backend/flask_api/")
from plotResultsSegEvaluation import create_filter_button

df.sort_values(by=["image_type", "reco", "dose"], inplace=True)
fig = px.line(df, x="reco", y="mse", color="project", 
                 facet_col="image_type", facet_row="lesion_type", title="Similarity MSE vs Reconstruction", template="simple_white")

fig.show()
#fig.write_html(os.path.join(ploting_dir, "sim_mse_vs_reco.html"))

In [12]:
df.sort_values(by=["image_type", "reco", "dose"], inplace=True)
fig = px.box(df, x="reco", y="ssim", color="project", 
                 facet_col="image_type", facet_row="lesion_type", title="Similarity SSIM vs Reconstruction", template="simple_white", points="all", hover_data=["dose"], range_y=[0,1])

fig.show()
#fig.write_html(os.path.join(ploting_dir, "sim_ssim_vs_reco.html"))

In [15]:
df.sort_values(by=["image_type", "reco", "dose"], inplace=True)
fig = px.box(df, x="reco", y="psnr", color="project", 
                 facet_col="image_type", facet_row="lesion_type", title="Similarity PSNR vs Reconstruction", template="simple_white", points="all", hover_data=["dose"], range_y=[0,50])

fig.show()
#fig.write_html(os.path.join(ploting_dir, "sim_psnr_vs_reco.html"))

In [ ]:
import os
import pandas as pd

dict_filename_imagetype = {'512x512x078_scale078_version3 (input scaled)': 'gan',
 'GAN589_Aly055_512x512x079_scale079_version3 (finale)': 'gan',
 'GAN_589_ALy055-1mm_LungTable_rem_5357_reg_scale079_512x512x079': 'gan',
 'old_gan_synth_Phm_origSize_20HU+_Intestin_Lung_Table_soft_211003_skaliert078_512x512x078': 'gan',
 'raw_512x512x078_scale078_version3 (input scaled)': 'raw',
 'raw_589_ALy055-1mm_LungTable_rem_5357_reg_scale079_512x512x079': 'raw',
 'raw_Aly055_512x512x079_scale079_version3 (finale)': 'raw',
 'raw_Phm_origSize_20HU+_Intestin_Lung_Table_soft_211003_skaliert078_512x512x078': 'raw'}

results_path = "../phantom_datasets/pred_print_templates_512x512"
df = pd.DataFrame()
for root, dirs, files in os.walk(results_path):
    for f in files:
        if f.endswith("results.csv"):
            meta_infos_path = root.split("/")[-1].split("_")
            #project = root.split("/")[-2]   
            df_temp = pd.read_csv(os.path.join(root, f))
            df_temp["project"] = meta_infos_path[1].split("-")[0]
            if meta_infos_path[2] == "3d":
                df_temp["network"] = f"{meta_infos_path[2]}_{meta_infos_path[3]}"
            else:
                df_temp["network"] = meta_infos_path[2]
            df_temp["lesion_type"] = meta_infos_path[1].split("-")[1]
            df_temp["image_type"] = df_temp["filename_pred"].apply(lambda x: dict_filename_imagetype[x[:-5]] if x.endswith('_mask') else dict_filename_imagetype[x])
            df = df.append(df_temp, ignore_index=True)

df_overall = df[df["lesion_id_gt"] == "all"]

In [ ]:

lesion_meta_paths = {
   '055': ['/home/uli/Insync/uligenske@gmail.com/Google Drive/Projekte_Uli/Deep_Learning/segmentation/projekte/phantom_datasets/230912_phantom238_055_unzipped_archive/meta/config/spheres.xlsx', "Version 3 (finale)"],
   '043': ['/home/uli/Insync/uligenske@gmail.com/Google Drive/Projekte_Uli/Deep_Learning/segmentation/projekte/phantom_datasets/230915_phantom241_043_unzipped_archive/meta/config/spheres.xlsx', "Version 3 final design"],
}

df_lesions = pd.DataFrame()
for phantom_name in ['043', '055']:
    df_phantom = df[df['project'] == phantom_name]
    lesion_meta_infos = pd.read_excel(lesion_meta_paths[phantom_name][0], 
                                      sheet_name=lesion_meta_paths[phantom_name][1])  
    df_lesions_temp = df_phantom[df_phantom['lesion_id_gt'] != 'all']
    df_lesions_temp = df_lesions_temp.groupby(['lesion_id_gt', 'network', 'filename_pred','image_type','lesion_type','project'], as_index=False).agg({'dice': ['max', 'sum']})
    df_lesions_temp.columns = ['_'.join(col) for col in df_lesions_temp.columns.values]
    df_lesions_temp.columns = [col[0:-1] if col[-1] == "_" else col for col in df_lesions_temp.columns.values]
    df_lesions_temp["lesion_id_gt_split"] = df_lesions_temp["lesion_id_gt"].apply(lambda x: x.split("_")[0])
    df_lesions_temp = df_lesions_temp.merge(lesion_meta_infos, left_on="lesion_id_gt_split", right_on="name")
    df_lesions_temp['dice'] = df_lesions_temp['dice_max']
    print(len(df_lesions_temp))

    df_lesions = df_lesions.append(df_lesions_temp)

# df_lesions.to_csv('results_csv/results_lesions_238_241_semi_old43.csv', index=False)


504
504


In [ ]:
df_lesions["lesion_type"].unique()

array(['planePhantom', 'spheres'], dtype=object)

In [ ]:
import plotly.express as px
from plot_results import create_filter_button

df_lesions_plot = df_lesions.sort_values(by=['lesion_type', 'image_type', 'network', 'R in mm', 'C in HU'])
df_lesions_plot['project+lesion_type'] = df_lesions_plot['project'] + '_' + df_lesions_plot['lesion_type']
fig = px.box(df_lesions_plot[df_lesions_plot["project+lesion_type"] == "043_planePhantom" ], 
             x="C in HU", y="dice", color="R in mm", facet_col="network", facet_row="image_type",
             points="all", template="simple_white" )
# create buttons
filter_button_colnames = ["project+lesion_type"]        
buttons = []
for filter_button in filter_button_colnames:
    for item in df_lesions_plot[filter_button].unique():
        button_temp = create_filter_button(df_lesions_plot, "C in HU", "dice", None, filter_button, item, "R in mm", "network", "image_type")
        buttons.append(button_temp)


fig.update_layout(
    updatemenus=[
        dict(
            direction="down",
            x=1,
            y=1.2,
            showactive=True,
            buttons=buttons,
            ),
        ],
    height=800,
    )
fig.update_yaxes(range=[0, 1])
fig.show()
fig.update_layout(
    height=None,
    )
fig.write_html(os.path.join(ploting_dir, "lesions_dice_templates.html"))

In [ ]:
import zipfile
import os


results_path = "results_csv/results_lesions_semi_old43_238_241_247_248.csv"
df_lesions = pd.read_csv(results_path)

df_lesions["phantom"] = df_lesions["phantom"].astype(str)
df_lesions.loc[df_lesions['phantom'] == '043old', 'phantom'] = '043_1'
df_lesions.loc[df_lesions['phantom'] == '241', 'phantom'] = '043_2'
df_lesions.loc[df_lesions['phantom'] == '238', 'phantom'] = '055_1'
df_lesions.loc[df_lesions['phantom'] == '247', 'phantom'] = '055_2'
df_lesions.loc[df_lesions['phantom'] == '248', 'phantom'] = '043_3'
df_lesions.loc[df_lesions['Reco+Kernel'] == 'first_body-sharp', 'Reco+Kernel'] = 'aice_body-sharp'
df_lesions.loc[df_lesions['Reco+Kernel'] == 'aice_body', 'Reco+Kernel'] = 'first_body'


df_lesions["phantom"].unique()

array(['055_1', '043_2', 'semi', '043_1', '055_2', '043_3'], dtype=object)

In [ ]:
df_lesions_dice_comparison_temp = df_lesions[df_lesions["phantom"].isin(["043_2", "043_3", "055_1", "055_2"])]
df_lesions_dice_comparison_temp = df_lesions_dice_comparison_temp[df_lesions_dice_comparison_temp["XRayTubeCurrent"] == 700]
df_lesions_dice_comparison_temp_summary = df_lesions_dice_comparison_temp.groupby(
    ["phantom",'Reco+Kernel', 'XRayTubeCurrent','network','CT', 'C in HU', 'R in mm', 'lesion_id_gt'], as_index=False
    ).agg({'dice': ['mean', 'std']})

df_lesions_dice_comparison_temp_summary.columns = ['_'.join(col) for col in df_lesions_dice_comparison_temp_summary.columns.values]
df_lesions_dice_comparison_temp_summary.columns = [col[0:-1] if col[-1] == "_" else col for col in df_lesions_dice_comparison_temp_summary.columns.values]

In [ ]:
import plotly.express as px
from plot_results import create_filter_button

df_lesions_dice_comparison_temp_summary = df_lesions_dice_comparison_temp_summary.sort_values(by=['network', 'R in mm', 'C in HU'])
df_lesions_dice_comparison_temp_summary['project+reco'] = df_lesions_dice_comparison_temp_summary['phantom'] + '_' + df_lesions_dice_comparison_temp_summary['Reco+Kernel']
#df_lesions_dice_comparison_temp_summary_best = df_lesions_dice_comparison_temp_summary.groupby('project+reco', 'R in mm', 'C in HU', 'network', 'CT', as_index=False).agg({'dice_mean': 'max'}) 
fig = px.box(df_lesions_dice_comparison_temp_summary[df_lesions_dice_comparison_temp_summary["project+reco"] == "043_2_aice_body-sharp"], 
             x="C in HU", y="dice_mean", color="R in mm", facet_col="network", facet_row="CT",
             points="all", template="simple_white")
# create buttons
filter_button_colnames = ["project+reco"]        
buttons = []
for filter_button in filter_button_colnames:
    for item in df_lesions_dice_comparison_temp_summary[filter_button].unique():
        button_temp = create_filter_button(df_lesions_dice_comparison_temp_summary, "C in HU", "dice_mean", None, filter_button, item, "R in mm", "network", "CT")
        buttons.append(button_temp)


fig.update_layout(
    updatemenus=[
        dict(
            direction="down",
            x=1,
            y=1.2,
            showactive=True,
            buttons=buttons,
            ),
        ],
    )
fig.update_yaxes(range=[0, 1])
fig.show()
# fig.update_layout(
#     height=None,
#     )
fig.write_html(os.path.join(ploting_dir, "lesions_dice_scans.html"))